In [1]:
from pynq import Overlay, allocate
import numpy as np
import struct

overlay = Overlay('v3.bit')
print("Loaded overlay!")
bnn_ip = overlay.feedforward_0

Loaded overlay!


In [2]:
IMG_SIZE = 28
IMG_SIZE_SQ = IMG_SIZE**2

with open('train-images.idx3-ubyte', 'rb') as f:
    magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
    images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows, cols)

with open('train-labels.idx1-ubyte', 'rb') as f:
    magic, num = struct.unpack(">II", f.read(8))
    labels = np.frombuffer(f.read(), dtype=np.uint8)

print(images.shape)
print(labels.shape)

image_data = np.reshape(images, (60000, IMG_SIZE_SQ))
# signed = np.where(flattened > 0, 1, -1)
# image_data = np.where(signed == 1, 0, 1).astype(np.int32)

print(image_data.shape)

(60000, 28, 28)
(60000,)
(60000, 784)


In [3]:
dma = overlay.axi_dma_0
dma_in = dma.sendchannel
dma_out = dma.recvchannel

In [4]:
input_buffer = allocate(shape=(IMG_SIZE_SQ,), dtype=np.int32)
output_buffer = allocate(shape=(10,), dtype=np.int32)

In [5]:
for i in range(10):
    actual_label = labels[i]
    np.copyto(input_buffer, image_data[i])
    
    dma_out.transfer(output_buffer)
    dma_in.transfer(input_buffer)
    
    bnn_ip.write(0x00, 0x81)

    dma_in.wait()
    dma_out.wait()

    result = np.array(output_buffer)
    print(result)
    predicted_label = np.argmax(result)
    print(f"The value predicted for this image is: {predicted_label} | expected: {actual_label} | {'success' if predicted_label == actual_label else 'failed'}")

[6 6 6 6 6 6 6 6 6 6]
The value predicted for this image is: 0 | expected: 5 | failed
[6 6 6 6 6 6 6 6 6 6]
The value predicted for this image is: 0 | expected: 0 | success
[6 6 6 6 6 6 6 6 6 6]
The value predicted for this image is: 0 | expected: 4 | failed
[6 6 6 6 6 6 6 6 6 6]
The value predicted for this image is: 0 | expected: 1 | failed
[6 6 6 6 6 6 6 6 6 6]
The value predicted for this image is: 0 | expected: 9 | failed
[6 6 6 6 6 6 6 6 6 6]
The value predicted for this image is: 0 | expected: 2 | failed
[6 6 6 6 6 6 6 6 6 6]
The value predicted for this image is: 0 | expected: 1 | failed
[6 6 6 6 6 6 6 6 6 6]
The value predicted for this image is: 0 | expected: 3 | failed
[6 6 6 6 6 6 6 6 6 6]
The value predicted for this image is: 0 | expected: 1 | failed
[6 6 6 6 6 6 6 6 6 6]
The value predicted for this image is: 0 | expected: 4 | failed
